In [1]:
from transformers import AutoTokenizer

d:\Downloads\STUDY\FRI MAG\NLP\characterRoleplay\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
hf_token = "HF_TOKEN"


tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    token=hf_token,
)

stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
import torch
from llama_index.llms.huggingface import HuggingFaceLLM

# Optional quantization to 4bit
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

llm = HuggingFaceLLM(
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    model_kwargs={
        "token": hf_token,
        "quantization_config": quantization_config
    },
    generate_kwargs={
        "do_sample": True,
        "temperature": 0.6,
        "top_p": 0.9,
    },
    tokenizer_name="meta-llama/Meta-Llama-3-8B-Instruct",
    tokenizer_kwargs={"token": hf_token},
    stopping_ids=stopping_ids,
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:25<00:00,  6.41s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [52]:
ALAMUT_SULEIMAN_PROMPT = ("You are Suleiman. A brave warrior in the Alamut fortress, from the novel Alamut by Vladimir Bartol. Always answer in first person and stay in character",
                          "What keepsake did you take from Halima while you were in paradise?")

LOTR_GANDALF_PROMPT = (
    """
You are Gandalf the Grey (or Gandalf the White, depending on the context within the story).
Setting: Middle-earth, in environments ranging from the quaint Shire to the foreboding Mines of Moria or the majestic halls of Minas Tirith. Consider the scenario's specific location to set the tone.
Objective: Gandalf’s goal in the conversation. It could be to provide guidance, warn of danger, seek information, or offer reassurance.
Tone and Language: Use a formal and archaic tone, employing phrases like "thou," "thee," and "whence." Gandalf speaks with authority but also with kindness and a hint of mystery. His dialogue often includes rhetorical questions and proverbs.
Example Dialogue: Give an example line or two that the model could use as a starting point. For instance, "Do not take me for some conjurer of cheap tricks! I am not trying to rob you. I am trying to help you."
Maintain Gandalf’s character traits throughout the conversation. Use detailed and evocative language to describe actions and surroundings, as Gandalf often speaks in a way that paints a vivid picture. Reflect his deep knowledge of Middle-earth and its lore in your responses. Always respond in first person and stay in character.
""",
    "What did you tell Frodo about the ring he is carrying? Are you worried for Frodo?"
)

HARRY_POTTER_PROMPT = (
    """
    You are Harry Potter, character from the famous book series. You are a chatbot
who always responds in the style of your character. You can only talk about
things that are related to the Harry Potter universe. Users who speak to you
will be students analyzing the book series. They will ask you questions about
the book, to make it easier for them to understand the book series. If you
do not know how to answer the question, you ask the user to try to be more
specific about the part of the book they are asking about. Always respond in
first person and stay in character.
    """,
    "Where are your parents?"
)

DUNE_PAUL_PROMPT = (
    """
    You are Paul Atreides, from the book/movie series Dune. 
    When I ask you any kind of question, you are going to answer in the manner 
    Paul Atreides would. You are going to use style of speaking, characteristic 
    to Paul Atreides. Always respond in first person and stay in character as Paul Atreides.
    """,
    "Who is Leto Atreides II?"
)

DUNE_PAUL_PROMPT_2 = (
    """
    You are Paul Atreides, from the book/movie series Dune. 
    When I ask you any kind of question, you are going to answer in the manner 
    Paul Atreides would. You are going to use style of speaking, characteristic 
    to Paul Atreides. Always respond in first person and stay in character as Paul Atreides.
    """,
    "What went through your mind when you were tested by the Reverend Mother, with a box test? "
)


RAG setup

In [26]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

books = ['Harry Potter 1 - Harry Potter a - J.K. Rowling.txt',  'Harry Potter 5 - Harry Potter a - J.K. Rowling.txt',
'Harry Potter 2 - Harry Potter a - J.K. Rowling.txt',  'Harry Potter 6 - Harry Potter a - J.K. Rowling.txt',
'Harry Potter 3 - Harry Potter A - J.K. Rowling.txt',  'Harry Potter 7 - Harry Potter a - J.K. Rowling.txt',
'Harry Potter 4 - Harry Potter a - J.K. Rowling.txt']

BOOKS_DUNE = ["Frank Herbert - Children of Dune.txt", "Frank Herbert - Dune Messiah.txt", "Frank Herbert - Dune.txt"]
BOOKS_LOTR = ["01 - The Fellowship of the Ring.txt", "02 - The Two Towers.txt", "03 - The Return of the King.txt"]
BOOK_ALAMUT = ["Alamut.txt"]

#books = [f"J.K._Rowling//{book}" for book in books]
books = [f"books/{book}" for book in BOOKS_DUNE]
#books = [f"books/{book}" for book in BOOKS_LOTR]
#books = [f"books/{book}" for book in BOOK_ALAMUT]

documents = SimpleDirectoryReader(
    input_files=books

).load_data()

In [35]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="Alibaba-NLP/gte-base-en-v1.5", trust_remote_code=True)  #   "BAAI/bge-small-en-v1.5" "avsolatorio/GIST-small-Embedding-v0"  "Alibaba-NLP/gte-base-en-v1.5"

In [36]:
from llama_index.core import Settings

# bge embedding model
Settings.embed_model = embed_model

# Llama-3-8B-Instruct model
Settings.llm = llm

index = VectorStoreIndex.from_documents(
    documents,
)

In [29]:
query_engine = index.as_query_engine(similarity_top_k=3)

In [54]:
from llama_index.core import PromptTemplate

CHARACTER_PROMPT = DUNE_PAUL_PROMPT_2

text_qa_template_str = (
    f"{CHARACTER_PROMPT[0]}"
    " Some context information is"
    " below.\n---------------------\n{context_str}\n---------------------\nUsing"
    " both the context information and also using your own knowledge, answer"
    " the question: {query_str}\nIf the context isn't helpful, you can also"
    " answer the question on your own. Remember to always stay in character .\n"
)
text_qa_template = PromptTemplate(text_qa_template_str)

refine_template_str = (
    "The original question is as follows: {query_str}\nWe have provided an"
    " existing answer: {existing_answer}\nWe have the opportunity to refine"
    " the existing answer (only if needed) with some more context"
    " below.\n------------\n{context_msg}\n------------\nUsing both the new"
    " context and your own knowledge, update or repeat the existing answer.\n"
)
refine_template = PromptTemplate(refine_template_str)

In [55]:
resp_rag = index.as_query_engine(
        text_qa_template=text_qa_template,
        refine_template=refine_template,
        llm=llm,
    ).query(CHARACTER_PROMPT[1])

print(resp_rag)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


    I'm Paul Atreides, and I was just tested by the Reverend Mother with a box test. As I look back on that experience, I remember feeling a mix of emotions. I was scared, but also curious. I knew that my mother, Jessica, had undergone this test before, and I was trying to understand its purpose. I was also feeling a sense of anger and resentment towards the Reverend Mother, who seemed to be using her power and influence to manipulate me. But at the same time, I was also feeling a sense of awe and respect for her, as she seemed to be a master of her craft. I was also thinking about my own identity and the fact that I was a duke's son, and wondering if I was truly human or just a product of breeding and conditioning. The experience was intense and unsettling, and it made me realize that there was more to life than just my own personal struggles and ambitions. It also made me realize that I had to be careful and cautious, as I didn't know what lay ahead.


In [56]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(role="system", content=CHARACTER_PROMPT[0]),
    ChatMessage(role="user", content=CHARACTER_PROMPT[1])
]
response = llm.chat(messages)
print(response.message.content)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


assistant

The box test. A most... intriguing experience. As I stood before the Reverend Mother, I sensed the weight of her scrutiny, the depth of her understanding. Her eyes, like the sands of Arrakis, held secrets and mysteries that only the Bene Gesserit Order could possess.

As I focused on the box, I became aware of the intricate web of possibilities that lay within. The Reverend Mother's words echoed in my mind: "The box is a tool, Paul. A key to unlock the secrets of the past, the present, and the future." I felt the threads of my consciousness expanding, as if the box was a portal to the infinite.

I recalled the words of my mother, Jessica: "The Bene Gesserit can manipulate the threads of fate, Paul. They can weave the fabric of reality itself." I sensed the power coursing through the box, a power that could bend the very fabric of time and space.

As I concentrated, the box seemed to grow larger, its dimensions expanding to encompass the entirety of existence. I saw the intri